In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import torch 
from torch import nn
from torch import tensor
import torch.nn.functional as F
import random
random.seed(69)

import pandas as pd
import random
import math
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder, MinMaxScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from tqdm import tqdm

def encoding_categorical_variables(data):

    all_cols = data.columns.tolist()

    encoder = OrdinalEncoder(categories=[['at_home', 'teacher', 'health','services','other']])
    data[['Mjob']] = encoder.fit_transform(data[['Mjob']]).astype('int64')
    data[['Fjob']] = encoder.fit_transform(data[['Fjob']]).astype('int64')

    encoder = OrdinalEncoder(categories=[['other', 'mother', 'father']])
    data[['guardian']] = encoder.fit_transform(data[['guardian']]).astype('int64')

    encoder = OrdinalEncoder(categories=[['home', 'reputation', 'course', 'other']])
    data[['reason']] = encoder.fit_transform(data[['reason']]).astype('int64')

    encoder = OrdinalEncoder(categories=[['no', 'yes']])
    data[['schoolsup']] = encoder.fit_transform(data[['schoolsup']]).astype('int64')
    data[['famsup']] = encoder.fit_transform(data[['famsup']]).astype('int64')
    data[['paid']] = encoder.fit_transform(data[['paid']]).astype('int64')
    data[['activities']] = encoder.fit_transform(data[['activities']]).astype('int64')
    data[['nursery']] = encoder.fit_transform(data[['nursery']]).astype('int64')
    data[['higher']] = encoder.fit_transform(data[['higher']]).astype('int64')
    data[['internet']] = encoder.fit_transform(data[['internet']]).astype('int64')
    data[['romantic']] = encoder.fit_transform(data[['romantic']]).astype('int64')

    categorical_cols = ['school', 'sex', 'address', 'famsize', 'Pstatus']
    categorical_data = data[categorical_cols]

    encoder = OneHotEncoder(sparse_output=False)
    encoded_data = encoder.fit_transform(categorical_data).astype('int64')

    data_encoded = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))

    numeric_cols = [col for col in all_cols if col not in categorical_cols]

    data = pd.concat([data_encoded, data[numeric_cols]], axis=1)

    # Droping column 'age' as G3 scores are independent of 'age'
    data.drop('age', axis=1, inplace=True)

    return data

def target_variable_encoding(data, data_test):

  # For binary classification tasks, according to the research paper, pass is considered if G3 >= 10, else fail
    for index, row in data.iterrows():
        if row['G3'] >= 13:
            data.at[index, 'G3'] = 1
        else:
            data.at[index, 'G3'] = 0

    for index, row in data_test.iterrows():
        if row['G3'] >= 13:
            data_test.at[index, 'G3'] = 1
        else:
            data_test.at[index, 'G3'] = 0

    return [data, data_test]

def test_train_split(data):

    # Out of 649 samples, 120 random samples are considered for the test set, and remaining ones for training.
    test_data_indices = random.sample(range(650), 120)

    data_test = data.iloc[test_data_indices].copy()
    data.drop(index=test_data_indices, inplace=True)

    data_test.reset_index(drop=True, inplace=True)
    data.reset_index(drop=True, inplace=True)

    return [data, data_test]

def feature_scaling(data, data_test):

  # All these columns are considered for feature scaling, to bring their value between 0 & 1
    feature_scaling_cols = ['Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures',
                          'famrel','freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'G1', 'G2']

    scaler = MinMaxScaler()

    data[feature_scaling_cols] = scaler.fit_transform(data[feature_scaling_cols])

    data_test[feature_scaling_cols] = scaler.transform(data_test[feature_scaling_cols])

    return [data, data_test]

def train_test_split_arrays(data, data_test):

    X_train = data.iloc[:, :-1].values

    # The target variable variable 'G3' is considered as 'y'
    y_train = data.iloc[:, -1:].values


    X_test = data_test.iloc[:, :-1].values
    y_test = data_test.iloc[:, -1:].values

    return [X_train, y_train, X_test, y_test]
    

In [2]:
data = pd.read_csv('student-por.csv', sep=",")
data = encoding_categorical_variables(data)
data, data_test = test_train_split(data)
data, data_test = feature_scaling(data, data_test)
data, data_test = target_variable_encoding(data, data_test)
x_train, y_train, x_test, y_test = train_test_split_arrays(data, data_test)
x = np.vstack((x_train, x_test))

In [3]:
def get_fairness_subset(X, predictions, indices):
    num_0 = 0 
    num_1 = 0 
    sum_0 = 0 
    sum_1 = 0 

    for i in indices:
        if(X[i][2]): 
            num_1 += 1 
            sum_1 += predictions[i] == 1 

        else:
            num_0 += 1 
            sum_0 += predictions[i] == 1 
    
    if(num_0 > 0 and num_1 > 0): return np.abs(sum_1/num_1 - sum_0/num_0)
    elif(num_0 == 0): return np.abs(sum_1/num_1)
    return np.abs(sum_0/num_0)
        

In [101]:
def fairness_set(subset):
    num_0 = 0 
    sum_0 = 0 
    num_1 = 0 
    sum_1 = 0

    num_1 = len(subset[0]) + len(subset[1]) 
    num_0 = len(subset[2]) + len(subset[3]) 
    sum_1 = len(subset[1])
    sum_0 = len(subset[3])
    
    if(num_0 > 0 and num_1 > 0): return np.abs(sum_1/num_1 - sum_0/num_0)
    elif(num_0 == 0): return np.abs(sum_1/num_1)
    return np.abs(sum_0/num_0)

In [126]:
model = LogisticRegression()
model.fit(x_train, y_train.ravel())
predictions = []
n = x.shape[0]
for i in range(n):
    predictions.append(model.predict(x[i].reshape(1,-1))[0])

In [127]:
X_female_one = set()
X_female_zero = set()
X_male_one = set()
X_male_zero = set()

for i in range(n):
    if(x[i][2]):
        if(predictions[i]): X_female_one.add(i)
        else: X_female_zero.add(i)
    else:
        if(predictions[i]): X_male_one.add(i)
        else: X_male_zero.add(i)

In [128]:
u_S = get_fairness_subset(x, predictions, range(n))
budget = 300
eps = 0.2 
best_subset = [set(), set(), set(), set()]

## New approach

In [129]:
group_1_0 = len(X_female_one) + len(X_male_zero) 
group_0_1 = len(X_female_zero) + len(X_male_one) 

In [130]:
# Assuming the budget is less than length of dataset/2 
done = 0 
if(group_1_0 > group_0_1):
    while(done < budget):
        if(len(X_female_one) > 0):
            new_el = random.sample(sorted(X_female_one), 1)[0]
            X_female_one.remove(new_el)
            best_subset[1].add(new_el) 
            done+=1

        if(len(X_male_zero) > 0):
            new_el = random.sample(sorted(X_male_zero), 1)[0]
            X_male_zero.remove(new_el)
            best_subset[2].add(new_el) 
            done+=1
        # print(done)

else:
    while(done < budget):
        if(len(X_female_zero) > 0):
            new_el = random.sample(sorted(X_female_zero), 1)[0]
            X_female_zero.remove(new_el)
            best_subset[0].add(new_el) 
            done+=1

        if(len(X_male_one) > 0):
            new_el = random.sample(sorted(X_male_one), 1)[0]
            X_male_one.remove(new_el)
            best_subset[3].add(new_el) 
            done+=1
        # print(done)

In [131]:
u_h = fairness_set(best_subset)

In [132]:
while np.abs(u_h - u_S > eps):

    if(group_1_0 > group_0_1):

        if(len(X_female_zero) > 0):
            new_el = random.sample(sorted(X_female_zero), 1)[0]
            X_female_zero.remove(new_el)
            best_subset[0].add(new_el) 

            remove_el = random.sample(sorted(best_subset[1]), 1)[0]
            best_subset[1].remove(remove_el)
            X_female_one.add(remove_el)

        if(len(X_male_one) > 0):
            new_el = random.sample(sorted(X_male_one), 1)[0]
            X_male_one.remove(new_el)
            best_subset[3].add(new_el) 

            remove_el = random.sample(sorted(best_subset[2]), 1)[0]
            best_subset[2].remove(remove_el)
            X_male_zero.add(remove_el)

    else:
        if(len(X_female_one) > 0):
            new_el = random.sample(sorted(X_female_one), 1)[0]
            X_female_one.remove(new_el)
            best_subset[1].add(new_el) 

            remove_el = random.sample(sorted(best_subset[0]), 1)[0]
            best_subset[0].remove(remove_el)
            X_female_zero.add(remove_el)

        if(len(X_male_zero) > 0):
            new_el = random.sample(sorted(X_male_zero), 1)[0]
            X_male_zero.remove(new_el)
            best_subset[2].add(new_el) 

            remove_el = random.sample(sorted(best_subset[3]), 1)[0]
            best_subset[3].remove(remove_el)
            X_male_one.add(remove_el)

    u_h = fairness_set(best_subset)


In [133]:
u_h, u_S

(0.39999999999999997, 0.2060896366241976)

## Old approach

In [117]:
done = 0 
best_subset = [set(), set(), set(), set()]

for i in range(min(len(X_female_one), budget - done)):
    new_el = random.sample(sorted(X_female_one), 1)[0]
    X_female_one.remove(new_el)
    best_subset[1].add(new_el) 
    done+=1 

for i in range(min(len(X_male_zero), budget - done)):
    new_el = random.sample(sorted(X_male_zero), 1)[0]
    X_male_zero.remove(new_el)
    best_subset[2].add(new_el) 
    done+=1  

for i in range(min(len(X_female_zero), budget - done)):
    new_el = random.sample(sorted(X_female_zero), 1)[0]
    X_female_zero.remove(new_el)
    best_subset[0].add(new_el) 
    done+=1

for i in range(min(len(X_male_one), budget - done)):
    new_el = random.sample(sorted(X_male_one), 1)[0]
    X_male_one.remove(new_el)
    best_subset[3].add(new_el) 
    done+=1 

In [118]:
u_h = fairness_set(best_subset)
print(u_h)

1.0


In [119]:
while np.abs(u_h - u_S > eps):
    if(len(X_female_zero) > 0): 
        remove_el = random.sample(sorted(best_subset[1]), 1)[0] 
        best_subset[1].remove(remove_el)
        X_female_one.add(remove_el)

        new_el = random.sample(sorted(X_female_zero), 1)[0]
        X_female_zero.remove(new_el)
        best_subset[0].add(new_el)

    elif(len(X_male_zero) > 0):
        remove_el = random.sample(sorted(best_subset[1]), 1)[0] 
        best_subset[1].remove(remove_el)
        X_female_one.add(remove_el)

        new_el = random.sample(sorted(X_male_zero), 1)[0]
        X_male_zero.remove(new_el)
        best_subset[2].add(new_el)

    elif(len(X_male_one) > 0):
        remove_el = random.sample(sorted(best_subset[1]), 1)[0] 
        best_subset[1].remove(remove_el)
        X_female_one.add(remove_el)

        new_el = random.sample(sorted(X_male_one), 1)[0]
        X_male_one.remove(new_el)
        best_subset[3].add(new_el)

    u_h = fairness_set(best_subset)
    print(u_h)

0.99
0.98
0.97
0.96
0.95
0.94
0.93
0.92
0.91
0.9
0.89
0.88
0.87
0.86
0.85
0.84
0.83
0.82
0.81
0.8
0.79
0.78
0.77
0.76
0.75
0.74
0.73
0.72
0.71
0.7
0.69
0.68
0.67
0.66
0.65
0.64
0.63
0.62
0.61
0.6
0.59
0.58
0.57
0.56
0.55
0.54
0.53
0.52
0.51
0.5
0.49
0.48
0.47
0.46
0.45
0.44
0.43
0.42
0.41
0.4


## Solving the company's objective

In [120]:
import torch
from torch.nn import Module
from torch import nn
from torch import tensor
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [121]:
def get_lengths(best_subset):
    return len(best_subset[0]), len(best_subset[1]), len(best_subset[2]), len(best_subset[3])

In [122]:
class logistic(nn.Module):
    def __init__(self, num_features):

        super().__init__()
        self.layer1 = nn.Linear(num_features, 1)

    def forward(self, x):
        return F.tanh(F.relu(self.layer1(x)))

In [123]:
def smooth_util(X, model, indices):
    num_0 = 0 
    num_1 = 0 
    sum_0 = 0 
    sum_1 = 0 
    for i in indices:
        
        if(X[i][2]): 
            num_1 += 1 
            sum_1 += model(tensor(X[i], dtype=torch.float))

        else:
            num_0 += 1 
            sum_0 +=  model(tensor(X[i], dtype=torch.float))

    return torch.abs(sum_1/num_1 - sum_0/num_0)

In [124]:
num_features = x.shape[1]
logistic_model = logistic(num_features)
opt = torch.optim.Adam(logistic_model.parameters(), lr = 1e-3)
l = 0.1

In [125]:
# pivot_params_weights = tensor(model.coef_, dtype = torch.float)
# pivot_params_bias = tensor(model.intercept_, dtype = torch.float)

In [126]:
# logistic_model.layer1.weight = nn.parameter.Parameter(pivot_params_weights)
# logistic_model.layer1.bias = nn.parameter.Parameter(pivot_params_bias)

In [127]:
# epochs = 1000
# dataloader = DataLoader(list(zip(x_train,y_train)), batch_size = 32, shuffle = True)

# for epoch in range(epochs):
#     total_loss = 0
#     total_num = 0 
#     num_classes = 2 
#     for data,label in dataloader: 

#         preds = logistic_model(tensor(data, dtype = torch.float))
#         one_hot_labels = F.one_hot(label, num_classes=num_classes).squeeze()
#         loss = F.binary_cross_entropy_with_logits(preds, tensor(one_hot_labels, dtype = torch.float))
#         total_loss += loss.item()
#         total_num += 1
#         loss.backward()
#         opt.step()

#     print(f"The epoch is {epoch+1} and the average loss is {total_loss/total_num}")

In [128]:
def project(linear_layer, pivot_params_weights, pivot_params_bias, l_norm):

    # weights = linear_layer.weight 
    # bias = linear_layer.bias 
    
    # new_weights = torch.clamp(weights, pivot_params_weights - l_norm, pivot_params_weights + l_norm)
    # new_bias = torch.clamp(bias, pivot_params_bias - l_norm, pivot_params_bias + l_norm)

    # linear_layer.weight = nn.parameter.Parameter(new_weights) 
    # linear_layer.bias = nn.parameter.Parameter(new_bias) 
    linear_layer.weight.data.clamp_(min=pivot_params_weights - l_norm, max=pivot_params_weights + l_norm)
    linear_layer.bias.data.clamp_(min=pivot_params_bias - l_norm, max=pivot_params_bias + l_norm)

In [131]:
trials = 10
for trial in range(trials):  

    with open('output.txt', 'a') as file:
        file.write(f"Trial {trial+1}\n")  
    logistic_model = logistic(num_features)
    opt = torch.optim.Adam(logistic_model.parameters(), lr = 1e-3)
    
    iterations = 5000
    logistic_model.train()
    for i in range(iterations):
        lag_term = 0
        for subset in best_subset:
            for s in subset:
                lag_term += torch.abs(logistic_model(tensor(x[s], dtype=torch.float)) - predictions[s])
        util = lag_term
        loss = -util 
        # loss = -lag_term
        # loss = -smooth_util(x, logistic_model, range(n))
        if(i % 500 == 0):
            with open('output.txt', 'a') as file:
                    file.write((str(lag_term.item())) + '\n')
        loss.backward()
        opt.step() 

    weights = logistic_model.layer1.weight.detach().clone()  # Shape: (out_features, in_features)
    bias = logistic_model.layer1.bias.detach().clone()     # Shape: (out_features,)

    # Combine the parameters into a single tensor
    pivot_params_weights = weights.flatten() 
    pivot_params_bias = bias

    with open('output.txt', 'a') as file:
        file.write('\n')

    iterations = 5000
    for i in range(iterations):
        fairness = smooth_util(x, logistic_model, range(n))
        util = fairness
        loss = -util 
        # loss = -lag_term
        # loss = -smooth_util(x, logistic_model, range(n))
        if(i % 500 == 0):
            with open('output.txt', 'a') as file:
                    file.write((str(fairness.item())) + '\n')
        loss.backward()
        opt.step() 
        project(logistic_model.layer1, pivot_params_weights, pivot_params_bias, 5)

    lag_term = 0
    for subset in best_subset:
        for s in subset:
            lag_term += torch.abs(logistic_model(tensor(x[s], dtype=torch.float)) - predictions[s])

    with open('output.txt', 'a') as file:
        file.write("Lagrangian term: " + str(lag_term.item()) + '\n')